### Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression,Lasso,Ridge,SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score,mean_squared_error
import pickle

### Define useful functions

In [508]:
def extract_zip_code(hotel_address):
    address_tmp = hotel_address.split(',')[-2].strip(' ').split(' ')[-1][:3]
    if address_tmp == 'NY1':
        return hotel_address.split(',')[-2].strip(' ').split(' ')[-1][2:5]
    else:
        return address_tmp

def extract_zip_code_5dig(hotel_address):
    address_tmp = hotel_address.split(',')[-2].strip(' ').split(' ')[-1][:5]
    if address_tmp == 'NY1':
        return hotel_address.split(',')[-2].strip(' ').split(' ')[-1][2:7]
    else:
        return address_tmp
    
def extract_price(hotel_room_price):
    room_price_tmp = hotel_room_price.strip('\r').strip('US$')
    if room_price_tmp == '':
        return None
    else:
        room_price_tmp = room_price_tmp.replace(',' , '')
        return float(room_price_tmp)
    
def extract_capacity(hotel_room_capacity):
    if pd.isnull(hotel_room_capacity):
        return None
    else:
        return int(hotel_room_capacity[-1])

def extract_chain(hotel_name):
    hotel_name_lower=hotel_name.lower()
    if 'doubletree by hilton' in hotel_name_lower:
        return 'Doubletree Hilton'
    elif 'homewood suites by hilton' in hotel_name_lower:
        return 'Homewood Suites Hilton'
    elif 'hilton garden' in hotel_name_lower:
        return 'Hilton Garden'
    elif 'home2' in hotel_name_lower:
        return 'Home2 Hilton'
    elif 'tru' in hotel_name_lower:
        return 'Tru Hilton'
    elif 'hilton' in hotel_name_lower:
        return 'Hilton'
    elif 'courtyard by marriott' in hotel_name_lower:
        return 'Courtyard Marriott'
    elif 'fairfield inn' in hotel_name_lower:
        return 'Fairfield Inn Marriott'
    elif 'residence inn by marriott' in hotel_name_lower:
        return 'Residence Inn Marriott'
    elif 'marriott' in hotel_name_lower:
        return 'Marriott'
    elif 'hyatt' in hotel_name_lower:
        return 'Hyatt'
    elif 'wyndham' in hotel_name_lower:
        return 'Wyndham'
    elif 'embassy' in hotel_name_lower:
        return 'Embassy'
    elif 'sheraton' in hotel_name_lower:
        return 'Sheraton'
    elif 'radisson' in hotel_name_lower:
        return 'Radisson'
    elif 'best western' in hotel_name_lower:
        return 'Best Western'
    elif 'hampton' in hotel_name_lower:
        return 'Hampton'
    elif 'holiday inn' in hotel_name_lower:
        return 'Holiday Inn'
    elif 'westin' in hotel_name_lower:
        return 'Westin'
    elif 'red roof' in hotel_name_lower:
        return 'Red Roof'
    else:
        return 'No chain'
       
def extract_hotel_stars(hotel_stars):
    if hotel_stars in ['1 stars', '1-star hotel']:
        return 1.0
    elif hotel_stars in ['1.5 stars', '1.5 star hotel']:
        return 1.5
    elif hotel_stars in ['2 stars', '2-star hotel']:
        return 2.0
    elif hotel_stars in ['2.5 stars', '2.5 star hotel']:
        return 2.5
    elif hotel_stars in ['3 stars', '3-star hotel']:
        return 3.0
    elif hotel_stars in ['3.5 stars', '3.5 star hotel']:
        return 3.5
    elif hotel_stars in ['4 stars', '4-star hotel']:
        return 4.0
    elif hotel_stars in ['4.5 stars', '4.5 star hotel']:
        return 4.5
    elif hotel_stars in ['5 stars', '5-star hotel']:
        return 5.0
    else:
        return None
    
def extract_cleanliness_rating(hotel_rating_breakdown):
    item_tmp = hotel_rating_breakdown.strip('[]').split('], [')[0]
    if item_tmp.split(',')[0]=="'Cleanliness'":
        return float(item_tmp.split(',')[1].strip(' ').strip("''"))
    else:
        return None
    
def extract_comfort_rating(hotel_rating_breakdown):
    item_tmp = hotel_rating_breakdown.strip('[]').split('], [')[1]
    if item_tmp.split(',')[0]=="'Comfort'":
        return float(item_tmp.split(',')[1].strip(' ').strip("''"))
    else:
        return None
    
def extract_location_rating(hotel_rating_breakdown):
    item_tmp = hotel_rating_breakdown.strip('[]').split('], [')[2]
    if item_tmp.split(',')[0]=="'Location'":
        return float(item_tmp.split(',')[1].strip(' ').strip("''"))
    else:
        return None
    
def extract_facilities_rating(hotel_rating_breakdown):
    item_tmp = hotel_rating_breakdown.strip('[]').split('], [')[3]
    if item_tmp.split(',')[0]=="'Facilities'":
        return float(item_tmp.split(',')[1].strip(' ').strip("''"))
    else:
        return None
    
def extract_staff_rating(hotel_rating_breakdown):
    item_tmp = hotel_rating_breakdown.strip('[]').split('], [')[4]
    if item_tmp.split(',')[0]=="'Staff'":
        return float(item_tmp.split(',')[1].strip(' ').strip("''"))
    else:
        return None
    
def extract_value_rating(hotel_rating_breakdown):
    item_tmp = hotel_rating_breakdown.strip('[]').split('], [')[5]
    if item_tmp.split(',')[0]=="'Value for money'":
        return float(item_tmp.split(',')[1].strip(' ').strip("''"))
    else:
        return None
    
def extract_wifi_rating(hotel_rating_breakdown):
    item_tmp = hotel_rating_breakdown.strip('[]').split('], [')
    if len(item_tmp)<7:
        return None
    elif item_tmp[6].split(',')[0]=="'Free WiFi'":
        return float(item_tmp[6].split(',')[1].strip(' ').strip("''"))
    else:
        return None
    
def display_info_hotel(df_hotels_item):
    print('Hotel name: ',df_hotels_item['hotel_name'].values[0])
    print('---------------------')
    print('Negative mentions per topic: ')
    print('(0) Room Comfort: ', str(int(df_hotels_item['0_neg'])))
    print('(1) Staff: ', str(int(df_hotels_item['1_neg'])))
    print('(2) Breakfast: ', str(int(df_hotels_item['2_neg'])))
    print('(3) Facilities: ', str(int(df_hotels_item['3_neg'])))
    print('(4) Location: ', str(int(df_hotels_item['4_neg'])))
    print('(5) Bathroom: ', str(int(df_hotels_item['5_neg'])))
    print('(6) Room Amenities: ', str(int(df_hotels_item['6_neg'])))
    print('(7) Bed Quality: ', str(int(df_hotels_item['7_neg'])))
    #print('Other/no main topic: ', str(int(df_hotels_item['-1_neg'])))
    
def change_topic_counts(df_hotel_item,counts_delta_vec): 
    df_hotel_item_cp = df_hotel_item.copy()
    df_hotel_item_cp.reset_index(drop=True,inplace=True)
    df_hotel_item_cp.at[0,'0_neg'] += counts_delta_vec[0]
    df_hotel_item_cp.at[0,'1_neg'] += counts_delta_vec[1]
    df_hotel_item_cp.at[0,'2_neg'] += counts_delta_vec[2]
    df_hotel_item_cp.at[0,'3_neg'] += counts_delta_vec[3]
    df_hotel_item_cp.at[0,'4_neg'] += counts_delta_vec[4]
    df_hotel_item_cp.at[0,'5_neg'] += counts_delta_vec[5]
    df_hotel_item_cp.at[0,'6_neg'] += counts_delta_vec[6]
    df_hotel_item_cp.at[0,'7_neg'] += counts_delta_vec[7]
    #df_hotel_item_cp.at[0,'sentences_count_neg'] +=sum(counts_delta_vec)
    df_hotel_item_cp[[str(n)+'_pc_neg' for n in range(-1,8)]] = 100*df_hotel_item_cp[[str(n)+'_neg' for n in range(-1,8)]].div(df_hotel_item_cp.sentences_count_neg, axis=0)
    return df_hotel_item_cp

def get_price_prediction(model,cat_feat,feat_compl_list,feat_compl_list_dum,df_hotel_item):
    input_features = df_hotel_item[feat_compl_list]
    input_target = df_hotel_item[['hotel_room_price_per_person_avg']]

    scaled_input = input_features.copy()
    scaled_input = scaled_input.reindex(columns = feat_compl_list_dum, fill_value=0)
    for cat in cat_feat:
        dummy_cat = cat+'_'+str(input_features[cat].values[0])
        if dummy_cat in scaled_input.columns:
            scaled_input.iloc[0, scaled_input.columns.get_loc(dummy_cat)] = 1
            
    price_pred = model.predict(scaled_input)[0][0]
    return price_pred

def update_price(model,cat_feat,feat_compl_list,feat_compl_list_dum,df_hotel_item,counts_delta_vec):
    # baseline price from model
    baseline_price = get_price_prediction(model,cat_feat,feat_compl_list,feat_compl_list_dum,df_hotel_item)
    # change negative mentions per topic
    df_hotel_item_changed = change_topic_counts(df_hotel_item,counts_delta_vec)
    # new price from model
    new_price = get_price_prediction(model,cat_feat,feat_compl_list,feat_compl_list_dum,df_hotel_item_changed)
    # per cent change
    per_cent_change = 100*(new_price-baseline_price)/baseline_price
    # return prices
    return [baseline_price,new_price,per_cent_change]

### Import datasets

In [509]:
# dataset of detailed info (room price, stars, ratings, location) for each hotel
df_hotel_detailed_info_list = pd.read_csv('./datasets/sample0_nys_hotel_detailed_info_list.csv',lineterminator='\n')
df_hotel_detailed_info_list_2 = pd.read_csv('./datasets/sample1_nys_hotel_detailed_info_list.csv',lineterminator='\n')
df_hotel_detailed_info_list_3 = pd.read_csv('./datasets/sample2_nys_hotel_detailed_info_list.csv',lineterminator='\n')

In [510]:
# dataset of number of negative mentions per topic for each hotel
df_negative_sentences_by_topic_pt = pd.read_csv('./datasets/df_negative_sentences_by_topic_pt_we.csv',lineterminator='\n')

### Cleaning and pre-processing

In [511]:
# rename room price, room name, and room capacity columns
df_hotel_detailed_info_list.rename(columns={'hotel_room_price\r': 'hotel_room_price'},inplace=True)
df_hotel_detailed_info_list_2.rename(columns={'hotel_room_price\r': 'hotel_room_price_2','hotel_room_name':'hotel_room_name_2','hotel_room_capacity':'hotel_room_capacity_2'},inplace=True)
df_hotel_detailed_info_list_3.rename(columns={'hotel_room_price\r': 'hotel_room_price_3','hotel_room_name':'hotel_room_name_3','hotel_room_capacity':'hotel_room_capacity_3'},inplace=True)

In [512]:
# extract zip codes in both 3-digit and 5-digit formats
df_hotel_detailed_info_list = df_hotel_detailed_info_list[~pd.isnull(df_hotel_detailed_info_list['hotel_address'])]
df_hotel_detailed_info_list['hotel_address_zip_code'] = df_hotel_detailed_info_list['hotel_address'].apply(extract_zip_code)
df_hotel_detailed_info_list['hotel_address_zip_code_5dig'] = df_hotel_detailed_info_list['hotel_address'].apply(extract_zip_code_5dig)

In [513]:
# extract room prices
df_hotel_detailed_info_list['hotel_room_price'] = df_hotel_detailed_info_list['hotel_room_price'].apply(extract_price)
df_hotel_detailed_info_list_2['hotel_room_price_2'] = df_hotel_detailed_info_list_2['hotel_room_price_2'].apply(extract_price)
df_hotel_detailed_info_list_3['hotel_room_price_3'] = df_hotel_detailed_info_list_3['hotel_room_price_3'].apply(extract_price)

In [514]:
# merge dataframes with room prices 2 and 3 onto main dataframe
df_hotel_detailed_info_list = df_hotel_detailed_info_list.merge(df_hotel_detailed_info_list_2[['hotel_url','hotel_room_price_2','hotel_room_name_2','hotel_room_capacity_2']],on='hotel_url')
df_hotel_detailed_info_list = df_hotel_detailed_info_list.merge(df_hotel_detailed_info_list_3[['hotel_url','hotel_room_price_3','hotel_room_name_3','hotel_room_capacity_3']],on='hotel_url')

In [515]:
# extract room capacities
df_hotel_detailed_info_list['hotel_room_capacity'] = df_hotel_detailed_info_list['hotel_room_capacity'].apply(extract_capacity)
df_hotel_detailed_info_list['hotel_room_capacity_2'] = df_hotel_detailed_info_list['hotel_room_capacity_2'].apply(extract_capacity)
df_hotel_detailed_info_list['hotel_room_capacity_3'] = df_hotel_detailed_info_list['hotel_room_capacity_3'].apply(extract_capacity)

In [516]:
# determine room prices per person
df_hotel_detailed_info_list['hotel_room_price_per_person'] = df_hotel_detailed_info_list['hotel_room_price']/df_hotel_detailed_info_list['hotel_room_capacity']
df_hotel_detailed_info_list['hotel_room_price_per_person_2'] = df_hotel_detailed_info_list['hotel_room_price_2']/df_hotel_detailed_info_list['hotel_room_capacity_2']
df_hotel_detailed_info_list['hotel_room_price_per_person_3'] = df_hotel_detailed_info_list['hotel_room_price_3']/df_hotel_detailed_info_list['hotel_room_capacity_3']

In [517]:
# drop hotels without room prices
df_hotel_detailed_info_list = df_hotel_detailed_info_list[~pd.isnull(df_hotel_detailed_info_list[['hotel_room_price_per_person','hotel_room_price_per_person_2','hotel_room_price_per_person_3']]).any(axis=1)]

In [518]:
# determine average room price per person for each hotel
df_hotel_detailed_info_list['hotel_room_price_per_person_avg'] = df_hotel_detailed_info_list[['hotel_room_price_per_person','hotel_room_price_per_person_2','hotel_room_price_per_person_3']].mean(axis=1)

In [519]:
# drop hotels without average room price
df_hotel_detailed_info_list = df_hotel_detailed_info_list[~pd.isnull(df_hotel_detailed_info_list['hotel_room_price_per_person_avg'])]

In [520]:
# extract hotel chain/brand
df_hotel_detailed_info_list['hotel_chain'] = df_hotel_detailed_info_list['hotel_name'].apply(extract_chain)

In [521]:
# extract aspect ratings
df_hotel_detailed_info_list['hotel_cleanliness_rating'] = df_hotel_detailed_info_list['hotel_rating_breakdown'].apply(extract_cleanliness_rating)
df_hotel_detailed_info_list['hotel_comfort_rating'] = df_hotel_detailed_info_list['hotel_rating_breakdown'].apply(extract_comfort_rating)
df_hotel_detailed_info_list['hotel_location_rating'] = df_hotel_detailed_info_list['hotel_rating_breakdown'].apply(extract_location_rating)
df_hotel_detailed_info_list['hotel_facilities_rating'] = df_hotel_detailed_info_list['hotel_rating_breakdown'].apply(extract_facilities_rating)
df_hotel_detailed_info_list['hotel_staff_rating'] = df_hotel_detailed_info_list['hotel_rating_breakdown'].apply(extract_staff_rating)
df_hotel_detailed_info_list['hotel_value_rating'] = df_hotel_detailed_info_list['hotel_rating_breakdown'].apply(extract_value_rating)
df_hotel_detailed_info_list['hotel_wifi_rating'] = df_hotel_detailed_info_list['hotel_rating_breakdown'].apply(extract_wifi_rating)

In [522]:
# median imputation for wifi rating (only a few missing values)
hotel_wifi_median = df_hotel_detailed_info_list['hotel_wifi_rating'].median()
df_hotel_detailed_info_list['hotel_wifi_rating'].fillna(hotel_wifi_median,inplace=True)

In [523]:
# extract hotel stars
df_hotel_detailed_info_list['hotel_stars'] = df_hotel_detailed_info_list['hotel_stars'].apply(extract_hotel_stars)

### Create copy of dataframe

In [524]:
df_hotel_detailed_info_list_filtered = df_hotel_detailed_info_list.copy()

### Remove hotels without stars

In [525]:
df_hotel_detailed_info_list_filtered = df_hotel_detailed_info_list_filtered[~pd.isnull(df_hotel_detailed_info_list_filtered['hotel_stars'])]

### Create column with standardized price per person by stars and location (5-digit zip code)

In [526]:
df_hotel_detailed_info_list_filtered['hotel_room_price_per_person_avg_grouped_mean'] = df_hotel_detailed_info_list_filtered.groupby(['hotel_stars','hotel_address_zip_code_5dig']).transform('mean')['hotel_room_price_per_person_avg']
df_hotel_detailed_info_list_filtered['hotel_room_price_per_person_avg_grouped_std'] = df_hotel_detailed_info_list_filtered.groupby(['hotel_stars','hotel_address_zip_code_5dig']).transform('std')['hotel_room_price_per_person_avg']

In [527]:
df_hotel_detailed_info_list_filtered = df_hotel_detailed_info_list_filtered[~pd.isnull(df_hotel_detailed_info_list_filtered['hotel_room_price_per_person_avg_grouped_std'])]

In [528]:
df_hotel_detailed_info_list_filtered['hotel_room_price_per_person_norm'] = df_hotel_detailed_info_list_filtered['hotel_room_price_per_person_avg']-df_hotel_detailed_info_list_filtered['hotel_room_price_per_person_avg_grouped_mean']
df_hotel_detailed_info_list_filtered['hotel_room_price_per_person_norm'] = df_hotel_detailed_info_list_filtered['hotel_room_price_per_person_norm'].div(df_hotel_detailed_info_list_filtered['hotel_room_price_per_person_avg_grouped_std'], axis=0)

In [529]:
df_hotel_detailed_info_list_filtered['hotel_room_price_per_person_pc_change'] = df_hotel_detailed_info_list_filtered['hotel_room_price_per_person_avg']-df_hotel_detailed_info_list_filtered['hotel_room_price_per_person_avg_grouped_mean']
df_hotel_detailed_info_list_filtered['hotel_room_price_per_person_pc_change'] = df_hotel_detailed_info_list_filtered['hotel_room_price_per_person_pc_change'].div(df_hotel_detailed_info_list_filtered['hotel_room_price_per_person_avg'], axis=0)

### Drop hotels with noisy room prices
(more than 30% away from average room price per person based on stars and location)

In [530]:
df_hotel_detailed_info_list_filtered = df_hotel_detailed_info_list_filtered[abs(df_hotel_detailed_info_list_filtered['hotel_room_price_per_person_pc_change'])<0.30]
#df_hotel_detailed_info_list_filtered = df_hotel_detailed_info_list_filtered[abs(df_hotel_detailed_info_list_filtered['hotel_room_price_per_person_norm'])<1.0]

### Join with topic count dataframe

In [531]:
neg_columns_to_rename = {'-1':'-1_neg','0':'0_neg','1':'1_neg','2':'2_neg','3':'3_neg','4':'4_neg','5':'5_neg',
                         '6':'6_neg','7':'7_neg',
                         '-1_pc':'-1_pc_neg','0_pc':'0_pc_neg','1_pc':'1_pc_neg','2_pc':'2_pc_neg',
                         '3_pc':'3_pc_neg','4_pc':'4_pc_neg','5_pc':'5_pc_neg',
                         '6_pc':'6_pc_neg','7_pc':'7_pc_neg',
                         'sentences_count':'sentences_count_neg'}

In [532]:
df_negative_sentences_by_topic_pt.rename(columns=neg_columns_to_rename,inplace=True)

In [533]:
df_hotel_detailed_info_list_filtered_merged = df_hotel_detailed_info_list_filtered.merge(df_negative_sentences_by_topic_pt,on='hotel_url')

### Remove hotels with zero topic counts across all topics (if any)

In [534]:
df_hotel_detailed_info_list_filtered_merged = df_hotel_detailed_info_list_filtered_merged[~(df_hotel_detailed_info_list_filtered_merged[['0_neg','1_neg','2_neg','3_neg','4_neg','5_neg','6_neg','7_neg']]==0.0).all(axis=1)]

### Select topic features to include in feature set
(and make them explicit percentages)

In [535]:
topic_features_list = ['0_pc_neg','1_pc_neg','2_pc_neg','3_pc_neg','4_pc_neg','5_pc_neg','6_pc_neg','7_pc_neg']
#topic_features_list = ['0_neg','1_neg','2_neg','3_neg','4_neg','5_neg','6_neg','7_neg','sentences_count_neg']

In [536]:
df_hotel_detailed_info_list_filtered_merged[topic_features_list] = 100*df_hotel_detailed_info_list_filtered_merged[topic_features_list]

### Print out complete dataset to csv

In [537]:
df_hotel_detailed_info_list_filtered_merged.to_csv('./input_app/df_hotels.csv',index=False)

### Print out complete dataset with unique names to csv for web app

In [538]:
df_hotels_unique_names = df_hotel_detailed_info_list_filtered_merged.copy()
df_hotels_unique_names['hotel_unique_name'] = df_hotels_unique_names['hotel_name'] + ' (' + df_hotels_unique_names['hotel_address_zip_code_5dig'] + ')'
df_hotels_unique_names.to_csv('./input_app/df_hotels_unique_names.csv',index=False)

## Modeling

### Create complete feature set and target

In [539]:
features_list1 = ['hotel_stars','hotel_overall_rating','hotel_address_zip_code_5dig','hotel_chain']
features_list2 = ['hotel_cleanliness_rating','hotel_comfort_rating','hotel_location_rating','hotel_facilities_rating','hotel_staff_rating','hotel_value_rating','hotel_wifi_rating']
features_complete_list = features_list1+features_list2+topic_features_list

df_features = df_hotel_detailed_info_list_filtered_merged[features_complete_list]
df_target = df_hotel_detailed_info_list_filtered_merged[['hotel_room_price_per_person_avg']]

In [540]:
# checking that feature set is correct
df_features.columns

Index(['hotel_stars', 'hotel_overall_rating', 'hotel_address_zip_code_5dig',
       'hotel_chain', 'hotel_cleanliness_rating', 'hotel_comfort_rating',
       'hotel_location_rating', 'hotel_facilities_rating',
       'hotel_staff_rating', 'hotel_value_rating', 'hotel_wifi_rating',
       '0_pc_neg', '1_pc_neg', '2_pc_neg', '3_pc_neg', '4_pc_neg', '5_pc_neg',
       '6_pc_neg', '7_pc_neg'],
      dtype='object')

### One-hot encoding for categorical features
(and standardization of numerical features if necessary)

In [541]:
categorical_features = ['hotel_address_zip_code_5dig','hotel_chain']

In [542]:
train_test_scaled_inputs = df_features.copy()

#train_test_num_features = train_test_scaled_inputs.drop(categorical_features,axis=1)
#no_scaling_features = topic_features_list
#train_test_num_scaled_features = train_test_num_features.drop(no_scaling_features,axis=1)
#train_test_num_scaled_features_col = train_test_num_scaled_features.columns.values
#scaler = StandardScaler()
#scaler.fit(train_test_num_scaled_features.values)
#train_test_num_scaled_features = scaler.transform(train_test_num_scaled_features.values)
#train_test_scaled_inputs[train_test_num_scaled_features_col] = train_test_num_scaled_features

In [543]:
train_test_scaled_inputs = pd.get_dummies(train_test_scaled_inputs,columns=categorical_features,drop_first=True)

### Train-validation split

In [544]:
# test_size=0.001 minimizes test set when training model for deployment
x_train, x_val, y_train, y_val = train_test_split(train_test_scaled_inputs,df_target, test_size = 0.001, random_state = 42)

### Linear regression

In [545]:
reg = LinearRegression()
reg.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [546]:
y_train_reg_pred = reg.predict(x_train)
y_val_reg_pred = reg.predict(x_val)

In [547]:
R2_train=reg.score(x_train,y_train)
adj_R2_train=1-(1-R2_train)*(x_train.shape[0]-1)/(x_train.shape[0]-x_train.shape[1]-1)

#R2_val=reg.score(x_val,y_val)
#adj_R2_val=1-(1-R2_val)*(x_val.shape[0]-1)/(x_val.shape[0]-x_val.shape[1]-1)

print(R2_train)
print(adj_R2_train)
#print()
#print(R2_val)
#print(adj_R2_val)

0.7938726563575038
0.723918401348683


In [548]:
# get intercept
const=reg.intercept_
print ('intercept = ',const)

# get weights for numerical features
reg_summary = pd.DataFrame(train_test_scaled_inputs.columns.values, columns=['Features'])
reg_summary['Weights'] = reg.coef_[0]
reg_summary.loc[0:16]

intercept =  [60.50735464]


,Features,Weights
0,hotel_stars,27.181526
1,hotel_overall_rating,-7.588442
2,hotel_cleanliness_rating,-15.222611
3,hotel_comfort_rating,22.020702
4,hotel_location_rating,0.476176
5,hotel_facilities_rating,21.446534
6,hotel_staff_rating,8.254040
7,hotel_value_rating,-28.790816
8,hotel_wifi_rating,1.064108
9,0_pc_neg,-0.765699


In [549]:
# plt.figure(figsize=(6,6))
# plt.scatter(y_train, y_train_reg_pred, 10, marker='.', c='r')
# plt.xlabel('Targets',size=14)
# plt.ylabel('Predictions',size=14)
# plt.xlim(0.0,300.0)
# plt.ylim(0.0,300.0)
# _ = plt.plot([0.0,300.0],[0.0,300.0])
# plt.show()

### Pickle model

In [550]:
regression_model_filename = 'price_regression_model.sav'
pickle.dump(reg, open(regression_model_filename, 'wb'))

### Pickle additional parameters

In [551]:
aux_info_1_filename = 'categorical_features.sav'
pickle.dump(categorical_features, open(aux_info_1_filename, 'wb'))

aux_info_2_filename = 'features_complete_list.sav'
pickle.dump(features_complete_list, open(aux_info_2_filename, 'wb'))

aux_info_3_filename = 'features_complete_list_dummies.sav'
pickle.dump(train_test_scaled_inputs.columns.values, open(aux_info_3_filename, 'wb'))

## Predictions by varing parameters
(testing the web app output)

### Import pickled files

In [552]:
reg_model = pickle.load(open('price_regression_model.sav','rb'))
cat_feats = pickle.load(open('categorical_features.sav','rb'))
feats_compl_list = pickle.load(open('features_complete_list.sav','rb'))
feats_compl_list_dummies = pickle.load(open('features_complete_list_dummies.sav','rb'))

### Input and test

In [558]:
input_hotel = df_hotels_unique_names[df_hotels_unique_names['hotel_unique_name']=='Ace Hotel New York (10001)']

In [559]:
display_info_hotel(input_hotel)

Hotel name:  Ace Hotel New York
---------------------
Negative mentions per topic: 
(0) Room Comfort:  36
(1) Staff:  17
(2) Breakfast:  15
(3) Facilities:  12
(4) Location:  15
(5) Bathroom:  34
(6) Room Amenities:  109
(7) Bed Quality:  26


In [571]:
counts_delta_vec = [0,0,0,0,0,-34,0,0]
test_output = update_price(reg_model,cat_feats,feats_compl_list,feats_compl_list_dummies,input_hotel,counts_delta_vec)

In [572]:
# original price, new price, % change
test_output

[171.21216900076604, 182.44499935270028, 6.560766338918341]

In [568]:
#input_hotel['hotel_room_price_per_person_avg']